# Unit Testing

Este notebook utiliza el algoritmo PLU desarrollado y desarrolla algunas  _pruebas unitarias_  utilizando la librería `pytest`.

En particular, se hace uso de los modulos desarrollados en las carpetas del repositorio nombradas "programacion" y "revision".

Algunas de las pruebas realizadas incluyen:
- `test_MatrizSimple`: prueba simple del algoritmo para una Matriz de 3x3
- `test_10Matrices`: prueba el algoritmo para 10 matrices, $A$, con dimensión aleatoria, con entradas de _floats_ aleatorias y se valida que $PA=LU$
- `test_comparaSciPyPLU_10Matrices`: para 10 matrices aleatorias se comparan los elementos $P$, $L$ y $U$ generados por el modulo propio y por la librería `SciPy`. 
- `test_bloques10Matrices` (**pendiente**): para 10 matrices aleatorias $A$, se ejecuta el algoritmo en su implementación por bloques y se valida que $PA=LU$.

_A continuación se describen los pasos para realizar estas pruebas dentro de un jupyter-notebook_

**Nota 1:** La presente implementación se puede ejecutar desde este cuaderno generado en `Google Collab` utilizando el botón de arriba.

**Nota 2:** La máquina o el _environment_ donde se ejecuta este ejemplo debe tener instalado `python >= 3.7`


>primero debemos intalar `pytest`

In [0]:
!pip install pytest

> Vamos a generar dos modulos que incluyen las funciones generadas en los notebooks contenidos en _programacion_ y _revision_.

Se van a generar el modulo `plu_functions.py` y `revision_functions.py` utilizando el _magic command_, `%file`

In [0]:
%%file plu_functions.py
# -*- coding: utf-8 -*-
'''
Extraído de Programacion/factorizacion_PLU.py
'''
import numpy as np
from scipy.linalg import solve_triangular


def forward_substitution(L, b):
    # lambda statements para facilitar escritura de ciclos
    def to_n(n): return np.arange(1, n + 1)
    def indexr(i): return i - 1
    '''
    Algoritmo de Forward substitution orientado a FILAS

    Esta función devuelve b para un sistema:
    Lx = b (1)
    ==========
    * Entradas:
        - L (array): matriz no singular, triangular inferior de nxn.
        - b (array): vector de nx1
    * Salidas:
        - y (array): vector de nx1, solución del sistema (1): Lx = b
    ==========
    Ejemplo:
        >>L = np.matrix([[1,0],[2,3]])
        >>b = np.array([2, 22])
        >>forward_substitution(L,b)
        > [2.0, 6.0]

    ==========
    Ref.:
    GCV - matrix computations (2013)
    Row-Oriented Forward Substitution (algorithm 3.1.1), p.106
    *********************
    Notas:
    Falta poner warnings por si el usuario mete inputs malos:
        ej: matriz no ciuadrada, matriz singular, las dimensiones de b
        y Y no coinciden
    '''
    n = len(b)
    y = np.zeros(n)
    y[indexr(1)] = b[indexr(1)] / L[indexr(1), indexr(1)]
    for i in np.arange(2, n + 1):
        suma = 0
        for j in to_n(i - 1):
            suma = suma + L[indexr(i), indexr(j)] * y[indexr(j)]
        y[indexr(i)] = (b[indexr(i)] - suma) / L[indexr(i), indexr(i)]

    return(y)


def backward_substitution(U, b):
    # lambda statements para facilitar escritura de ciclos
    def to_n(n): return np.arange(1, n + 1)
    def indexr(i): return i - 1
    '''
    Algoritmo de Backward substitution orientado a FILAS

    Esta función devuelve b para un sistema:
    Lx = b (1)
    ==========
    * Entradas:
        - U (array): matriz no singular, triangular superior de nxn.
        - b (array): vector de nx1
    * Salidas:
        - y: vector de nx1, solución del sistema (1): Ux = b
    ==========
    Ejemplo:
        >>U = np.matrix([[1, 2],[0,3]])
        >>b = np.array([49, 21])
        >>BackwardSubsRow(U,b)
        > array([35.,  7.])

    ==========
    Ref.:
    GCV - matrix computations (2013)
    Row-Oriented Backward Substitution (algorithm 3.1.2), p.107
    '''
    n = len(b)
    x = np.zeros(n)
    x[indexr(n)] = b[indexr(n)] / U[indexr(n), indexr(n)]
    for i in np.arange(1, n - 1 + 1)[::-1]:
        suma = 0
        for j in np.arange(i + 1, n + 1):
            suma = suma + U[indexr(i), indexr(j)] * x[indexr(j)]
        x[indexr(i)] = (b[indexr(i)] - suma) / U[indexr(i), indexr(i)]

    return(x)


def get_P(piv):
    # lambda statements para facilitar escritura de ciclos
    def to_n(n): return np.arange(1, n + 1)
    def indexr(i): return i - 1
    '''
    Esta función obtiene la matriz pivote derivada del intercambio de elementos
    en la matriz identidad original
    ==========
    * Entradas:
        - p: índices
    * Salidas:
        - P (matriz): matriz de permutación de nxn

    '''
    n = len(piv) + 1
    P = np.eye(n)
    for j in to_n(n - 1):
        aux = P[indexr(j), :].copy()
        P[indexr(j), :] = P[indexr(piv[indexr(j)]), :].copy()
        P[indexr(piv[indexr(j)]), :] = aux.copy()

    return(P)


def PLU_factor(A):
    '''
    Esta función desarrolla la factorizacióón PA = LU, donde P es la matriz de
    permutación codificada por piv(l:n - 1), guarda los ííndices fila de los
    pivotes, de tal modo que la columnas intercambiadas se guardan en el vector
    P.
    Esta función devuelve P en el sistema:
    Lx = b (1)
    ==========
    * Entradas:
        - A: array de nxn.
    * Salidas:
        - P (vector): nx1, con los ííndices de las columnas intercambiadas en
        el pivoteo.
        - L (matriz): matriz triangular inferior de nxn
        - U (matriz): matriz triangular superior nxn
    ==========
    Ejemplo:
        >>A = np.array([[2, 2, 3], [-4, -4, -3], [4, 8, 3]])
        >>P, L, U = PLU_factor(A)
        >>P
        >array([[0., 1., 0.],
       [0., 0., 1.],
       [1., 0., 0.]])
        >>L
        >array([[ 1. ,  0. ,  0. ],
       [-1. ,  1. ,  0. ],
       [-0.5,  0. ,  1. ]])
        >>U
        >array([[-4. , -4. , -3. ],
       [ 0. ,  4. ,  0. ],
       [ 0. ,  0. ,  1.5]])
       >>np.matmul(P, A)==np.matmul(L, U)
       >array([[ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True]])
    '''

    # lambda statements para facilitar escritura de ciclos
    def to_n(n): return np.arange(1, n + 1)
    def indexr(i): return i - 1
    # inicialización de elementos
    A = A.astype('float64')
    n = A.shape[0]
    L = np.eye(n)
    U = np.zeros((n, n))
    piv = np.arange(1, n - 1 + 1)
    v = np.zeros(n)

    for j in to_n(n):
        if j == 1:
            v = A[indexr(j):n, indexr(j)].copy()
        else:
            a = A[:, indexr(j)].copy()
            for k in to_n(j - 1):
                aux = a[indexr(k)].copy()
                a[indexr(k)] = a[indexr(piv[indexr(k)])].copy()
                a[indexr(piv[indexr(k)])] = aux.copy()
            z = forward_substitution(
                L[indexr(1):(j - 1), indexr(1):(j - 1)], a[indexr(1):(j - 1)])
            U[indexr(1):(j - 1), indexr(j)] = z.copy()
            v[indexr(j):n] = (a[indexr(j):n] -
                              np.matmul(L[indexr(j):n, indexr(1):(j - 1)], z)).copy()

        if j < n:
            mu = (np.argmax(np.abs(v[indexr(j):n])) + j).copy()
            piv[indexr(j)] = mu.copy()
            aux = v[indexr(j)].copy()
            v[indexr(j)] = v[indexr(mu)].copy()
            v[indexr(mu)] = aux.copy()
            if v[indexr(j)] != 0:
                L[indexr(j + 1):n, indexr(j)
                  ] = (v[indexr(j + 1):n] / v[indexr(j)]).copy()
            if j > 1:
                aux = L[indexr(j), indexr(1):(j - 1)].copy()
                L[indexr(j), indexr(1):(j - 1)] = L[indexr(mu),
                                                    indexr(1):(j - 1)].copy()
                L[indexr(mu), indexr(1):(j - 1)] = aux.copy()
        U[indexr(j), indexr(j)] = v[indexr(j)].copy()

    P = get_P(piv)

    return P, L, U


def simulate(n, times):
    error_abs = np.zeros(times)
    for i in to_n(times):
        np.random.seed(i)
        A = np.random.normal(0, 1, (n, n))
        x_real = np.random.normal(0, 1, n)
        b = np.matmul(A, x_real)
        x_est = solve(A, b)
        error_abs[indexr(i)] = np.sum(np.abs(x_real - x_est))

    return(np.mean(error_abs))


'''
Funciones Extraidas de Programacion/LU solve.ipynb
'''


def LUdecomp(a):
    n = len(a)
    for k in range(0, n - 1):
        for i in range(k + 1, n):
            if a[i, k] != 0.0:
                lam = a[i, k] / a[k, k]
                a[i, k + 1:n] = a[i, k + 1:n] - lam * a[k, k + 1:n]
                a[i, k] = lam
    return a


def LUsolve(a, b):
    n = len(a)
    for k in range(1, n):
        b[k] = b[k] - np.dot(a[k, 0:k], b[0:k])
    b[n - 1] = b[n - 1] / a[n - 1, n - 1]
    for k in range(n - 2, -1, -1):
        b[k] = (b[k] - np.dot(a[k, k + 1:n], b[k + 1:n])) / a[k, k]
    return b


'''
Funciones Extraidas de Programacion/EliminacionPorBloques.ipynb
'''
# from math import ceil
# import src.algorithms.FactorizacionPLU as plu
# from src.algorithms import EliminacionPorBloques as block
# from src.algorithms import FactorizacionPLU
# import src
# import sys
# sys.path.append('./../../')
#
# # OPTIONAL: Load the "autoreload" extension so that code can change
# %load_ext autoreload
#
# # OPTIONAL: always reload modules so that as you change code in src, it gets loaded
# %autoreload 2
#
# eliminador = block.EliminacionPorBloques()
#
# A = np.array([[1.,  4., -2., -5.],
#               [-3.,  9.,  8.,  7.],
#               [5.,  1., -6., -4.],
#               [6., -1.,  2.,  8.]])
#
# b = np.array([3, 0, 7, 6, 9], dtype=np.float)
# a = np.array([3, 0, 7, 6], dtype=np.float)
#
# x_prueba = np.linalg.solve(A, a)
# print(x_prueba)
#
# x_nuestra = eliminador.solve_blocks(A, a)
# print(x_nuestra)
#
#
# # -----
#
# # Cambiar en el futuro
# def nuestro_algoritmo(A, b):
#     return np.linalg.solve(A, b)
#     #factorizador = plu.FactorizacionPLU()
#     # return factorizador.solve(A,b)
#
#
# def solve_blocks(A, b):
#     # Check that it is a squared matrix
#     A_column = A.shape[1]
#     if A_column == A.shape[0]:
#         if A_column % 2 == 0:
#             x = int(A_column / 2)
#         else:
#             x = int(ceil(A_column / 2))
#         print(x)
#         A11 = A[:x, :x]
#         A12 = A[:x, x:]
#         A21 = A[x:, :x]
#         A22 = A[x:, x:]
#
#         print(A11)
#         print(A12)
#         print(A21)
#         print(A22)
#
#         b1 = b[:x]
#         b2 = b[x:]
#
#         A11_b1 = nuestro_algoritmo(A11, b1)
#         A11_A12 = nuestro_algoritmo(A11, A12)
#
#         print(A11_b1, A11_A12)
#         Schur = A22 - A21@A11_A12
#         b_hat = b2 - A21@A11_b1
#
#         x2 = nuestro_algoritmo(Schur, b_hat)
#         x1 = nuestro_algoritmo(A11, (b1 - A12@x2))
#
#         X = np.block([x1, x2])
#
#     else:
#         X = -2
#         print("Please enter a squared matrix")
#     return X
#
# # ----
#
#
# # def por_bloques(A, b):
# B = np.array([[1.,  4., -2., -5., 1],
#               [-3.,  9.,  8.,  7., 2],
#               [5.,  1., -6., -4., 3],
#               [5.,  1., -6., -4., 4],
#               [6., -1.,  2.,  8., 5]])
#
# A = np.array([[1.,  4., -2., -5.],
#               [-3.,  9.,  8.,  7.],
#               [5.,  1., -6., -4.],
#               [6., -1.,  2.,  8.]])
#
# b = np.array([3, 0, 7, 6, 9], dtype=np.float)
# a = np.array([3, 0, 7, 6], dtype=np.float)
#
# x_prueba = np.linalg.solve(A, a)
# print(x_prueba)
#
# x_nuestra = solve_blocks(A, a)
# print(x_nuestra)
#
# x_prueba = np.linalg.solve(B, b)
# print(x_prueba)
#
# x_nuestra = solve_blocks(B, b)
# print(x_nuestra)


Writing plu_functions.py


In [0]:
ls

Rcf.py  sample_data/


In [0]:
%%file revision_functions.py
# -*- coding: utf-8 -*-
import numpy as np
import pprint
import pandas as pd
import time
#import plu_functions
import plu_functions


def crea_matrices(dim_limite_inf, dim_limite_sup, entradas_lim_sup):
    '''
    Esta función crea matrices de forma aleatoria. Se le debe especificar el límite 
    inferior y superior de la dimensión de la matriz y el límite inferior y superior 
    de los números de entrada a la matriz. La función devuelve una matriz A cuadrada 
    de dimensión nxn:
    ==========
    * Entradas:
        - dim_limite_inf (integer): número entero que corresponde a la dimensión 
        mínima de la matriz
        - dim_liminte_sup (integer): número entero que corresponde a la dimensión 
        máxima de la matriz
        - entradas_lim_inf (integer): número entero que corresponde al número más
        chico que puede tener la matriz como entrada
        - entradas_lim_sup (integer): número entero que corresponde al número más 
        grande que puede tener la matriz como entrada
    * Salidas:
        - A (matriz): matriz cuadrada de dimensión nxn
    ==========
    Ejemplo:
        >> dim_limite_inf = 2
        >> dim_liminte_sup = 10^4
        >> entradas_lim_inf = -99
        >> entradas_lim_sup = 99
        >> crea_matrices(dim_limite_inf,dim_limite_sup,
        entradas_lim_inf,entradas_lim_sup)
        > A
        >array([[-28, -50,  79],
                 [-22,  87, -23],
                 [  0, -97,  22]])
    '''
    n = np.random.randint(dim_limite_inf, dim_limite_sup)
    #A=np.array(np.random.randint(entradas_lim_inf,entradas_lim_sup, size=(n, n)))
    A = entradas_lim_sup * \
        np.random.random_sample((n, n)) - 2 * entradas_lim_sup
    return A, n


def factoriza_plu(A):
    '''
    Esta función ejecuta el algoritmo de PLU y mide el tiempo de ejecución total 
    del algoritmo. Se le debe especificar una matriz cuadrada A. La función 
    devuelve el tiempo total en segundos de ejecución del algoritmo, 2 matrices 
    (L,U) y un vector P.

    * Para ver más información sobre la función factorización_PLU, favor de 
    consultar su documentación.
    ==========
    * Entradas:
        - A (matriz): matriz cuadrada de dimensión nxn
    * Salidas:
        - P (vector): nx1, con los índices de las columnas intercambiadas en el 
        pivoteo.
        - L (matriz): matriz triangular inferior de nxn
        - U (matriz): matriz triangular superior nxn
        - timempo_total (float): tiempo total en segundos que tarda la ejecución
         del algoritmo PLU.
    ==========
    Ejemplo:
        >>A = np.array([[2, 2, 3], [-4, -4, -3], [4, 8, 3]])
        >>factoriza_plu(A)
        >P
        >array([[0., 1., 0.],
               [0., 0., 1.],
               [1., 0., 0.]])
        >L
        >array([[ 1. ,  0. ,  0. ],
               [-1. ,  1. ,  0. ],
               [-0.5,  0. ,  1. ]])
        >U
        >array([[-4. , -4. , -3. ],
               [ 0. ,  4. ,  0. ],
               [ 0. ,  0. ,  1.5]])
        >tiempo_total
        0.001711
    '''
    start_time = time.time()
    P, L, U = plu_functions.PLU(A)
    end_time = time.time()
    tiempo_total = end_time - start_time
    return tiempo_total, P, L, U


def solve_A_b(A, b):
    '''
    Esta función ejecuta el algoritmo que resuelve un sistema de ecuaciones de la
    forma Ax = b con la factorización PLU y mide el tiempo de ejecución total del 
    algoritmo. Se le debe especificar una matriz cuadrada A y un vector de tamaño nx1.
     La función devuelve el tiempo total en segundos de ejecución del
    algoritmo y un vector x de tamaño nx1 con la solución del sistema de ecuaciones.

    * Para ver más información sobre la función solve, favor de consultar su documentación.
    ==========
    * Entradas:
        - A (matriz): matriz cuadrada de dimensión nxn
        - b (vector): vector de nx1 con las soluciones del lado derecho del = de la ecuación
    * Salidas:
        - x_est (vector): vector de nx1 con la solución del sistema de ecuaciones
    ==========
    Ejemplo:
        >>A = np.array([[2, 2, 3], [-4, -4, -3], [4, 8, 3]])
        >>b = np.array([1, 3, 2])
        >>solve_A_b(A)
        >x_est
        >array([-3.25,  1.25, 1.66])
        >tiempo_total
        0.000956
    '''
    start_time = time.time()
    x_est = plu_functions.solve(A, b)
    end_time = time.time()
    tiempo_total = end_time - start_time
    return tiempo_total, x_est


def condicion(A):
    '''
    Esta función calcula la condición de la matriz cuadrada A. La condición sirve 
    para ver cómo se comportará el algoritmo ante pequeñas perturbaciones en x.
    ==========
    * Entradas:
        - A: matriz cuadrada de dimensión nxn
    * Salidas:
        - cond (float): condicion de la matriz A
    ==========
    Ejemplo:
        >>A = np.array([[2, 2, 3], [-4, -4, -3], [4, 8, 3]])
        >>condicion(A)
        >cond
        13.882903
    '''
    cond = np.linalg.cond(A)
    return cond


Writing revision_functions.py


verificamos que se hayan escrito los modulos en la ruta donde está el notebook.

In [0]:
ls

También debemos crear un script en `python` que nos va a permitir generar los test en `pytest`. El nombre de este script debe empezar con "test_".

In [0]:
%%file test_plu.py
# -*- coding: utf-8 -*-
import numpy as np
import pprint
import pytest
import scipy

import plu_functions
import revision_functions as rv


def test_MatrizSimple():
    '''
    Evalúa la factorización PLU para la matriz simple A
    Para más información del ejemplo ver el sript de "programacion_PLU.py"
    '''
    A = np.array([[2, 2, 3], [-4, -4, -3], [4, 8, 3]])
    P, L, U = plu_functions.PLU_factor(A)

    print("Matriz a probar")
    pprint.pprint(A)
    print("P:")
    pprint.pprint(P)
    print("L:")
    pprint.pprint(L)
    print("U:")
    pprint.pprint(U)
    print("PA=LU")
    # probar que PA=LU
    assert (np.matmul(P, A) == np.matmul(L, U)).all()

def test_10Matrices():
    '''
    Evalúa la factorización PLU para varias matrices cuadradas aleatorias con
    dimensión entre 2x2 y (10^3)x(10^3).
    NOTA: Las entradas de los números son floats están entre -10.000 y 10.000.
    Para más información del ejemplo ver el sript de "revision_PLU.py"
    '''
    np.random.seed(3338014) # semilla para replicabilidad de las pruebas

    eps = 1.0E-8 # para definir la precisión a 8 dígitos
    n = 10
    dim_lim_inf = 2
    dim_lim_sup = 10
    ents_lim_sup = 10000
    # crea matrices aleatorias
    for i in range(1, n + 1):
        mat, _ = rv.crea_matrices(dim_lim_inf, dim_lim_sup, ents_lim_sup)
    # prueba el algoritmo PLU
        P, L, U = plu_functions.PLU_factor(mat)
        # verifica que la diferencia entre los elementos de la operación PA-LU
        # sean menores a eps
        assert ((np.matmul(P, mat) - np.matmul(L, U))).all() <= eps

def test_comparaSciPyPLU_10Matrices():
    '''
    Compara los elementos P, L, y U de la factorización asociada obtenidos con
    la librería SciPy y con la librería creada para este ejercicio.
    Evalúa que la diferencia de cada elemento de cada una de dichas matrices no
    sea mayor a "eps".
    '''
    np.random.seed(3338014) # semilla para replicabilidad de las pruebas

    eps = 1.0E-8 # para definir la precisión a 8 dígitos
    n = 10
    dim_lim_inf = 2
    dim_lim_sup = 10**3
    ents_lim_sup = 10000
    # crea matrices aleatorias
    for i in range(1, n + 1):
        mat, _ = rv.crea_matrices(dim_lim_inf, dim_lim_sup, ents_lim_sup)

        # prueba el algoritmo PLU
        P, L, U = plu_functions.PLU_factor(mat)
        P_scipy, L_scipy, U_scipy = scipy.linalg.lu(mat)
        # verificar que los elementos de los dos modulos son iguales
        print ((P_scipy==P).all() and (L_scipy==L).all() and (U_scipy==U).all())


Overwriting test_plu.py


In [0]:
ls

plu_functions.py  revision_functions.py  test_plu.py
__pycache__/      sample_data/


Ahora es posible ejecutar los tres test que van hasta el momento ejecutanto `pytest` desde la línea de comandos (usando _magic commands_).

In [0]:
!pytest -v test_plu.py

============================= test session starts ==============================
platform linux2 -- Python 2.7.17, pytest-3.6.4, py-1.8.0, pluggy-0.7.1 -- /usr/bin/python2
cachedir: .pytest_cache
rootdir: /content, inifile:
collected 3 items                                                              

test_plu.py::test_MatrizSimple PASSED                                    [ 33%]
test_plu.py::test_10Matrices PASSED                                      [ 66%]
test_plu.py::test_comparaSciPyPLU_10Matrices PASSED                      [100%]

=========================== 3 passed in 0.36 seconds ===========================
